## Proyecto Naive Bayes: *Clasificación de reseñas de Google*

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

### 1. Observacion de los datos

In [6]:
df =  pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv')
df

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0
...,...,...,...
886,com.rovio.angrybirds,loved it i loooooooooooooovvved it because it...,1
887,com.rovio.angrybirds,all time legendary game the birthday party le...,1
888,com.rovio.angrybirds,ads are way to heavy listen to the bad review...,0
889,com.rovio.angrybirds,fun works perfectly well. ads aren't as annoy...,1


In [7]:
df.drop('package_name', axis = 1, inplace = True)

In [9]:
df.head()

,review,polarity
0,privacy at least put some option appear offli...,0
1,"messenger issues ever since the last update, ...",0
2,profile any time my wife or anybody has more ...,0
3,the new features suck for those of us who don...,0
4,forced reload on uploading pic on replying co...,0


In [10]:
df["review"] = df["review"].str.strip().str.lower()

In [41]:
# Función para limpiar el texto
def clean_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres no ASCII
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Eliminar caracteres especiales adicionales
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.strip()

# Aplicar limpieza de texto
df["review"] = df["review"].apply(clean_text)

*Conclusiones preliminares*:
- *Decidimos que no es necesario hacer un EDA, ya que no tiene sentido, pues la única variable que nos interesa es la que contiene texto.*
- *Hacemos un procesado del texto: eliminamos espacios, mayúsculas, caracteres especiales y numéricos.*

### 2. Entrenamiento del modelo Naive Bayes Multinomial

In [42]:
X_train, X_test, y_train, y_test = train_test_split(df.review, df.polarity, test_size=0.2, random_state=63)

In [43]:
vec_model = CountVectorizer(stop_words = "english")
X_train = vec_model.fit_transform(X_train)
X_test = vec_model.transform(X_test)

In [46]:
# Obtener la bolsa de palabras
vocabulary = vec_model.get_feature_names_out()

# Convertir la lista a una cadena y imprimirla
print('Bolsa de palabras que se usa para el entrenamiento:')
print(', '.join(vocabulary))

Bolsa de palabras que se usa para el entrenamiento:
aa, aafnaii, aakhirat, aalikati, aap, aaps, aapsssssss, aaru, abilities, ability, abke, able, aboutwants, absolute, absolutely, accecesible, accepted, acces, accesible, access, accessed, accessible, accessing, accident, accidently, according, accordion, account, accounts, accurate, achievements, achive, act, action, actions, active, activities, activity, acts, actual, actually, ad, add, added, addicted, addicting, addiction, addictive, adding, addition, additional, address, adds, adequate, adfree, adjust, adjustment, admin, ads, adsand, advanced, advantage, adventurous, advertisement, advertisements, advertising, adverts, advice, advised, aesthetics, affect, affirm, aficionado, afloat, againuntil, age, agenda, agent, ages, aggresively, ago, agree, ahead, aint, air, akash, album, albumcamera, albums, alert, alerts, ali, allahsubhanahutallah, allow, allowed, allows, allready, alot, alphabetical, alright, alterations, alternative, altoge

In [73]:
# Probando que el vectorizador funciona adecuadamente

print(f'Reseña de prueba: {X_test[0]}')
print('\nPalabras del conjunto de entrenamiento que también aparecen en la reseña de test junto con su aparición:\n')
for i, cont in enumerate(X_test.toarray()[0]):
  if cont!=0:
    print(f'Palabra: "{vec_model.get_feature_names_out()[i]}"')
    print(f'--> Aparece {cont} veces en la reseña.')

Reseña de prueba:   (0, 344)	1
  (0, 540)	1
  (0, 1070)	2
  (0, 1218)	1
  (0, 2173)	1
  (0, 2765)	1

Palabras del conjunto de entrenamiento que también aparecen en la reseña de test junto con su aparición:

Palabra: "browser"
--> Aparece 1 veces en la reseña.
Palabra: "come"
--> Aparece 1 veces en la reseña.
Palabra: "firefox"
--> Aparece 2 veces en la reseña.
Palabra: "gonna"
--> Aparece 1 veces en la reseña.
Palabra: "platform"
--> Aparece 1 veces en la reseña.
Palabra: "standard"
--> Aparece 1 veces en la reseña.


In [74]:
# Modelo sin optimizar

model = MultinomialNB().fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85       126
           1       0.67      0.55      0.60        53

    accuracy                           0.79       179
   macro avg       0.75      0.72      0.73       179
weighted avg       0.78      0.79      0.78       179



- *La clase 0 tiene un rendimiento mejor que la clase 1, con una precisión de 0.82 frente a un 0.67. Esto podría deberse a que la clase 1 representa únicamente el 30% de las muestras totales, por lo que existe un desbalance de clases.*
- *Vamos a probar a asignar pesos a las clases favoreciendo a la clase minoritaria, a ver si podemos mejorarlo.*

In [75]:
# Modelo con pesos asignados

class_prior = {0: 0.3, 1: 0.7}

model_w = MultinomialNB().fit(X_train, y_train, sample_weight=[class_prior[i] for i in y_train])

y_pred = model_w.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.79      0.85       126
           1       0.63      0.83      0.72        53

    accuracy                           0.80       179
   macro avg       0.77      0.81      0.78       179
weighted avg       0.83      0.80      0.81       179



In [76]:
# Búsqueda de hiperparámetros para mejorar las métricas

from sklearn.model_selection import GridSearchCV
# Define el espacio de búsqueda de hiperparámetros
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],  # Valores para el parámetro de suavizado
    'fit_prior': [True, False],      # Valores para el parámetro de ajuste previo
}

# Inicializa el clasificador Naive Bayes Multinomial
nb_classifier = MultinomialNB()

# Inicializa el objeto GridSearchCV
grid_search = GridSearchCV(estimator=nb_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Realiza la búsqueda de hiperparámetros en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Muestra los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Obtiene la mejor configuración del modelo
best_nb_model = grid_search.best_estimator_

# Realiza predicciones en el conjunto de prueba utilizando el mejor modelo
y_pred = best_nb_model.predict(X_test)

# Evalúa el rendimiento del mejor modelo
print("Informe de clasificación del mejor modelo:")
print(classification_report(y_test, y_pred))

Mejores hiperparámetros encontrados:
{'alpha': 2.0, 'fit_prior': False}
Informe de clasificación del mejor modelo:
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       126
           1       0.70      0.53      0.60        53

    accuracy                           0.79       179
   macro avg       0.76      0.72      0.73       179
weighted avg       0.78      0.79      0.78       179



In [77]:
# Definir las probabilidades a priori para cada clase
class_prior = {0: 0.3, 1: 0.7}
# Paso 4: Entrenamiento del modelo

model_w = MultinomialNB(alpha = 0.3).fit(X_train, y_train, sample_weight=[class_prior[i] for i in y_train])

# Realizar predicciones en el conjunto de prueba
y_pred = model_w.predict(X_test)

# Evaluar el rendimiento del modelo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.85      0.87       126
           1       0.68      0.75      0.71        53

    accuracy                           0.82       179
   macro avg       0.78      0.80      0.79       179
weighted avg       0.83      0.82      0.82       179



*Conclusiones finales:*

- *Asignando pesos a las clases y probando ciertos valores de hiperparámetros hemos conseguido encontrar una versión mucho mejor de este modelo bayesiano que la que viene por default.*

- *Es posible que probando otros modelos como Random Forest o XGBoost nos dé mejores resultados en un futuro.*

In [78]:
# Guardado del modelo
from joblib import dump
dump(model_w, 'modelo_entrenado_naivebayes.joblib')

['modelo_entrenado_naivebayes.joblib']